# Ain't No Mountain High Enough

The cipher is made of 5 keys, `K1` to `K5`, which are 5×5 matrixes.
It can be summed up by the equation: `enc(M) = K5 * K4 * K3 * K2 * K1 * M`.
Additionally, we can replace any key `Ki` by the flag `FL` and request encryption.

In [1]:
N = 5
P = 251
K = GF(P)

In [2]:
def pad(msg, n):
    # Bytestrings only pls
    if type(msg) == str:
        msg = msg.encode()
    # Apply random padding
    while (len(msg) % (n*n) != 0):
        msg += os.urandom(1)
    # Matrixfy and return
    return [matrix([list(i[j:j+n]) for j in range(0,n*n,n)]).change_ring(K)
            for i in [msg[k:k+n*n] for k in range(0,len(msg),n*n)]]

In [3]:
def encMC(msg, key):
    # Convert msg to matrix
    if type(msg) in { str, bytes, list }:
        msg = pad(msg, N)

    # For all nxn msg matrices
    ct = []
    for mi in msg:
        # Hill-cipher encrypt with every key matrix
        for i in range(N):
            mi = key[i] * mi
        # Add result to ciphertext
        ct += [mi]

    # Return ciphertext in hex
    return ct

A very important property of this cipher is that if `E = enc(M)`, then `enc(M * X) = E * X`. This means that if we set `M = Id` we can just ask the server once for `E = enc(Id)` and do everything locally afterward.

In [4]:
M = MatrixSpace(K, N)
m = M.identity_matrix()

In [5]:
msg = ''.join(map(chr, m.list()))
msg

'\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01'

The data was gathered using [`get_data.py`](./get_data.py).

In [6]:
data = [
    b'|   [134, 135, 128,  42, 181]\n',
    b'|   [ 82,   6,  60, 212, 103]\n',
    b'|   [104, 130,  29,  12,  77]\n',
    b'|   [167, 118, 190,  17, 162]\n',
    b'|   [198,  73,  95, 219, 178]\n',
]

craw = pad(sum([eval(x[1:].strip().decode()) for x in data], []), N)[0]
# craw = K5 * K4 * K3 * K2 * K1

In [7]:
data = [
    b'|   [176, 106, 194,  92,  83]\n',
    b'|   [142, 143, 149,   2, 233]\n',
    b'|   [150,  52,  87, 154, 177]\n',
    b'|   [ 77, 219,  13, 235, 112]\n',
    b'|   [ 73,  83,   8, 245, 166]\n',
    
    b'|   [152, 233,  74, 169, 143]\n',
    b'|   [  6,  66,  71,  46, 169]\n',
    b'|   [117,  98,  54, 241, 166]\n',
    b'|   [209, 149,  25,  62,  45]\n',
    b'|   [236,  44, 101, 137, 244]\n',
    
    b'|   [100, 214,  47,  28, 217]\n',
    b'|   [ 72, 220, 189, 143,  92]\n',
    b'|   [200,  73, 165,  12, 138]\n',
    b'|   [ 99, 139, 222,  89, 131]\n',
    b'|   [106, 214, 149,  94, 186]\n',
    
    b'|   [ 97,  77, 208,  34, 239]\n',
    b'|   [191, 236,  13,   8,  53]\n',
    b'|   [ 69, 229, 141, 248, 132]\n',
    b'|   [233, 152, 185, 119, 192]\n',
    b'|   [132,  77,  69,  76,  78]\n',
    
    b'|   [ 30,  19,  95, 220, 219]\n',
    b'|   [153,  17, 137, 245, 215]\n',
    b'|   [ 44,  16,  87,   6,  86]\n',
    b'|   [ 66, 111, 173, 207,  64]\n',
    b'|   [139,   7,  65, 249, 233]\n',
]

cflag = pad(sum([eval(x[1:].strip().decode()) for x in data], []), N)
# cflag = [
#     FL * K4 * K3 * K2 * K1,
#     K5 * FL * K3 * K2 * K1,
#     K5 * K4 * FL * K2 * K1,
#     K5 * K4 * K3 * FL * K1,
#     K5 * K4 * K3 * K2 * FL,
# ]

In our quest to retrieve `FL`, we might try to multiply the content of `cflag` by `craw^-1` in order to remove the noise at the right.

We would get the following entries:
```
(1) FL * K5^-1
(2) K5 * FL * K4^-1 * K5^-1
(3) K5 * K4 * FL * K3^-1 * K4^-1 * K5^-1
(4) K5 * K4 * K3 * FL * K2^-1 * K3^-1 * K4^-1 * K5^-1
(5): K5 * K4 * K3 * K2 * FL * craw^-1
```

It is interesting to see that the trailing noise of `(i)` is the inverse of the leading noise of `(i+1)`:
```
  (1) * (2) * (3) * (4) * (5)
= (FL * K5^-1) * (K5 * FL * K4^-1 * K5^-1) * (3) * (4) * (5)
= FL^2 * K4^-1 * K5^-1 * (K5 * K4 * FL * K3^-1 * K4^-1 * K5^-1) * (4) * (5)
= ...
= FL^5 * craw^-1
```

If we multiply this quantity by `craw`, we will thus know `FL^5`.

In [8]:
FL5 = prod(cf/craw for cf in cflag) * craw
FL5

[ 62 170 162 116  44]
[ 95 160 148 199 165]
[174 159 192   9 105]
[ 38  24 216 115 108]
[198  13  17 201  51]

Knowing `X^5`, how can we know `X`? When `X = P*D*P^-1` with `D` a diagonal matrix, `X^(1/5) = P*D^(1/5)*P^-1`. Hence we can know one 5th root of `X` by searching fifth roots of the diagonal of `D`.

In [9]:
D, P = FL5.diagonalization()

In [10]:
all_diag = [d.nth_root(5, all=True) for d in D.diagonal()]
all_diag

[[129, 139, 70, 19, 145],
 [143, 193, 99, 95, 223],
 [101, 31, 12, 118, 240],
 [228, 234, 42, 162, 87],
 [105, 154, 92, 68, 83]]

In [12]:
from itertools import product

for diag in product(*all_diag):
    FL = P * M.diagonal_matrix(diag) * P^-1
    flag = ''.join(chr(x%251) for x in FL.list())
    if flag.isascii():
        print(flag)

n0_m0uNtAin_2_hIGh_F0R_u!
